In [3]:
import pandas as pd
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import json
import gzip
import pickle 
import numpy as np
from array import array

# Code for proprocessing of text 
### Used for tokenization and other preprocessing

In [4]:
#for stopword removal
nltk.download('stopwords')
stop = set(stopwords.words('english') + list(string.punctuation))


# Function to tokenize and preprocess a document
def preprocess(text):
    tokens = set(word_tokenize(text.lower())) # get all tokens
    return [i for i in tokens if i not in stop] # get all tokens without stopwords

    # add lematization later

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Creation of ininverted index

In [5]:
try:
    import orjson as _json
    loads = _json.loads
except Exception:
    import json as _json
    loads = _json.loads

In [31]:
OUTPUT_PATH = "inverted_index.jsonl"

#create terms used in each document
#extend with count
corpus = 'trec-tot-2025-corpus.jsonl.gz'

print("Starting with the creation of the inverted index (6,407,814 entries)") # based on the number on the website

inverted_index = defaultdict(lambda: array('I'))
with gzip.open(corpus, 'rt', encoding='utf-8') as f_in, \
    open('inverted_index.pkl', 'wb') as f_out:
    for line_number, line in enumerate(f_in, start=1):
        line = line.strip()
        if not line:
            continue  # skip empty lines
        try:
            data = loads(line)
            text = data.get("text", "")
            tokens = preprocess(text)
            tokens = set(tokens)                         #one for all later ad counts as well
            for token in tokens:
                inverted_index[token].append(np.int32(line_number))
        except json.JSONDecodeError as e:
            print(f"[Line {line_number}] Invalid JSON: {e}")

        if line_number % 10000 == 0: # ginve information of progress and saving it to a file
            print(line_number)
        if line_number % 100000 == 0:
            inverted_index_np = {term: np.array(ids, dtype=np.int32) for term, ids in inverted_index.items()}
            pickle.dump(inverted_index_np, f_out)
            f_out.flush()
    pickle.dump(dict(inverted_index), f_out) # saving it
    f_out.flush()


print(f"Finished creation of the inverted index.")

Starting with the creation of the inverted index (6,407,814 entries)
10000


KeyboardInterrupt: 

In [6]:
OUTPUT_PATH = "inverted_index.jsonl"

#create terms used in each document
#extend with count
corpus = 'trec-tot-2025-corpus.jsonl.gz'

print("Starting with the creation of the inverted index (6,407,814 entries)") # based on the number on the website

inverted_index = defaultdict(lambda: array('I'))
with gzip.open(corpus, 'rt', encoding='utf-8') as f_in, \
    open('inverted_index.pkl', 'wb') as f_out:
    for line_number, line in enumerate(f_in, start=1):
        line = line.strip()
        if not line:
            continue  # skip empty lines
        try:
            data = loads(line)
            text = data.get("text", "")
            tokens = preprocess(text)
            tokens = set(tokens)                         #one for all later ad counts as well
            for token in tokens:
                inverted_index[token].append(line_number)
        except json.JSONDecodeError as e:
            print(f"[Line {line_number}] Invalid JSON: {e}")

        if line_number % 10000 == 0: # ginve information of progress and saving it to a file
            print(line_number)
        if line_number % 100000 == 0:
            inverted_index_np = {term: np.array(ids, dtype=np.int32) for term, ids in inverted_index.items()}
            pickle.dump(inverted_index_np, f_out)
            f_out.flush()
    pickle.dump(dict(inverted_index), f_out) # saving it
    f_out.flush()


print(f"Finished creation of the inverted index.")

Starting with the creation of the inverted index (6,407,814 entries)
10000
20000
30000
40000


KeyboardInterrupt: 

# Opening the inverted index

In [ ]:
with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

print(inverted_index)